In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features
import ipynb.fs.full.features_distancias as f_distancias

df_train = pd.read_csv('./data/train_filtrado.csv')
# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

df_train = features.llenar_nulls(df_train)
df_test = features.llenar_nulls(df_test, hgb_mean=True, df_fill=df_train)

# df_train, df_test = features_de_csvs(df_train, df_test)

# df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.15)

In [2]:
df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

df_train_idf = pd.read_csv('./data/train_idf.csv')
df_test_idf = pd.read_csv('./data/test_idf.csv')

df_train_f = pd.merge(df_train_f, df_train_idf, on= 'id', how= 'left')
df_test_f = pd.merge(df_test_f, df_test_idf, on= 'id', how= 'left')

df_train_f = f_distancias.feature_distancias(df_train_f)
df_test_f = f_distancias.feature_distancias(df_test_f, df_train_f)

## LightGBM model

In [3]:
from sklearn.model_selection import train_test_split

class LightGBMWrapper(lgb.LGBMRegressor):
    
    def fit(self, x, y):        
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.08363501292068126)
        return super(LightGBMWrapper, self).fit(x_train, y_train)
    
    def predict(self, X):
        return super(LightGBMWrapper, self).predict(X, 
               num_iteration=self.best_iteration_)

hps = {'bagging_fraction': 0.8994254451454451,
 'bagging_freq': 90.0,
 'feature_fraction': 0.22390222216361155,
 'learning_rate': 0.09425558913069614,
 'max_depth': 16.0,
 'num_leaves': 179.0,
 'test_size': 0.09437441103433714}

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae', # Si se deja vacio se toma el ideal para llegar al 'objective'
    'num_leaves': int(hps['num_leaves']),
    'learning_rate': hps['learning_rate'],
    'feature_fraction': hps['feature_fraction'],
    'bagging_fraction': hps['bagging_fraction'],
    'bagging_freq': int(hps['bagging_freq']),
    'max_depth': int(hps['max_depth']),
    'verbose': 0
}

lgb_m = LightGBMWrapper(**params)

## Keras model

In [4]:
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation

def keras_modelo():    
    model = Sequential()
    model.add(BatchNormalization())
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=1, activation='linear'))
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'], validation_split=0.1)
    return model

keras_m = KerasRegressor(build_fn=keras_modelo, epochs=10)

Using TensorFlow backend.


## XGBoost

In [7]:
import xgboost as xgb

class XGBoostWrapper(xgb.XGBRegressor):
    
    def fit(self, x, y):
        return super(xgb.XGBRegressor, self).fit(x, y, early_stopping_rounds=2, eval_metric='mae', eval_set=[(x, y)])
    
    def predict(self, X):
        return super(xgb.XGBRegressor, self).predict(X)


hps = {'alpha': 20.91434940058063,
       'colsample_bytree': 0.65,
       'learning_rate': 0.14,
       'max_depth': int(16.0),
       'n_estimators': int(150.0),
       'test_size': 0.2,
       'early_stopping_rounds': 5,
       'n_jobs': 2}


n_estimators = int(hps['n_estimators'])
max_depth = int(hps['max_depth'])

xgb_m = XGBoostWrapper(**hps)

## Stacking

In [8]:
from sklearn.preprocessing import MinMaxScaler

from vecstack import StackingTransformer

features = ['antiguedad', 'metroscubiertos', 'metrostotales', 'lat', 'lng',
       'diferencia_metros', 'dias_desde_datos', 'tam_ambientes',
       'promedio_precio_ciudad', 'varianza_precio_ciudad', 'promedio_id_zona',
       'promedio_id_zona_gen', 'varianza_id_zona', 'count_id_zona',
       'promedio_precio_tipo_propiedad_ciudad',
       'promedio_precio_tipo_propiedad_ciudad_gen',
       'count_tipo_propiedad_ciudad',
       'promedio_precio_habitaciones_banos_garages',
       'promedio_precio_hbg_tipo_propiedad',
       'promedio_precio_hbg_tipo_propiedad_provincia', 'puntaje', 'idf_titulo',
       'idf_descripcion', 'distancia_ciudad_centrica']



x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.001)

modelos = [('lightgbm', lgb_m), 
            #('keras', keras_m), 
           ('xgboost', xgb_m)]

stack = StackingTransformer(modelos, regression=True, verbose=2)

stack = stack.fit(x_train, y_train)

s_train = stack.transform(x_train)
s_test = stack.transform(x_test)

task:         [regression]
metric:       [mean_absolute_error]
variant:      [A]
n_estimators: [2]

estimator  0: [lightgbm: LightGBMWrapper]
    fold  0:  [513655.93763197]
    fold  1:  [510078.32774160]
    fold  2:  [507999.36682222]
    fold  3:  [514485.21685609]
    ----
    MEAN:     [511554.71226297] + [2637.38495861]

estimator  1: [xgboost: XGBoostWrapper]
[19:36:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:2.17582e+06
Will train until validation_0-mae hasn't improved in 2 rounds.
[1]	validation_0-mae:1.87867e+06
[2]	validation_0-mae:1.62333e+06
[3]	validation_0-mae:1.40435e+06
[4]	validation_0-mae:1.21565e+06
[5]	validation_0-mae:1.05436e+06
[6]	validation_0-mae:915778
[7]	validation_0-mae:797172
[8]	validation_0-mae:695769
[9]	validation_0-mae:609336
[10]	validation_0-mae:535973
[11]	validation_0-mae:474013
[12]	validation_0-mae:421600
[13]	validation_0-mae:378232
[14]	validatio

[98]	validation_0-mae:58065.2
[99]	validation_0-mae:57459.9
[100]	validation_0-mae:57118.9
[101]	validation_0-mae:56836.6
[102]	validation_0-mae:56407.8
[103]	validation_0-mae:55901.6
[104]	validation_0-mae:55564.4
[105]	validation_0-mae:55322.6
[106]	validation_0-mae:55265.9
[107]	validation_0-mae:55135.4
[108]	validation_0-mae:54971.2
[109]	validation_0-mae:54675
[110]	validation_0-mae:53942.3
[111]	validation_0-mae:53897.9
[112]	validation_0-mae:53444.3
[113]	validation_0-mae:53052.4
[114]	validation_0-mae:52481
[115]	validation_0-mae:52130.5
[116]	validation_0-mae:51921.1
[117]	validation_0-mae:51563.6
[118]	validation_0-mae:51282
[119]	validation_0-mae:51192.5
[120]	validation_0-mae:51062.9
[121]	validation_0-mae:50941.1
[122]	validation_0-mae:50794
[123]	validation_0-mae:50438.5
[124]	validation_0-mae:50153.2
[125]	validation_0-mae:49944.4
[126]	validation_0-mae:49765.2
[127]	validation_0-mae:49709.3
[128]	validation_0-mae:49436.8
[129]	validation_0-mae:49342.4
[130]	validation_0

[56]	validation_0-mae:81470.1
[57]	validation_0-mae:80459.3
[58]	validation_0-mae:79783.8
[59]	validation_0-mae:78653.5
[60]	validation_0-mae:77761.4
[61]	validation_0-mae:76803.9
[62]	validation_0-mae:75731.9
[63]	validation_0-mae:74520.1
[64]	validation_0-mae:73928.9
[65]	validation_0-mae:73534.4
[66]	validation_0-mae:72592.6
[67]	validation_0-mae:71631.1
[68]	validation_0-mae:71251
[69]	validation_0-mae:70877.4
[70]	validation_0-mae:70355.5
[71]	validation_0-mae:70256.7
[72]	validation_0-mae:70093.9
[73]	validation_0-mae:69747.9
[74]	validation_0-mae:68824.4
[75]	validation_0-mae:68016.6
[76]	validation_0-mae:67388.9
[77]	validation_0-mae:67090.3
[78]	validation_0-mae:66272.6
[79]	validation_0-mae:65712.4
[80]	validation_0-mae:65011.9
[81]	validation_0-mae:64345.8
[82]	validation_0-mae:63992.6
[83]	validation_0-mae:63502.2
[84]	validation_0-mae:63119.8
[85]	validation_0-mae:61872.6
[86]	validation_0-mae:61696.5
[87]	validation_0-mae:61073.8
[88]	validation_0-mae:60617.7
[89]	validat

In [10]:
s_train = stack.transform(utils.filtrar_features(df_train_f.drop('precio', axis=1), features))
s_test = stack.transform(utils.filtrar_features(df_test_f, features))

Transforming...

estimator  0: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

Transforming...

estimator  0: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE



## Prediccion con todos los features + stacking

In [11]:
df_train_s = df_train_f.copy()
df_test_s = df_test_f.copy()

df_train_s['stack01'], df_train_s['stack02'] = zip(*s_train)
df_test_s['stack01'], df_test_s['stack02'] = zip(*s_test)

In [12]:
df_train_s['id'] = df_train['id']
df_test_s['id'] = df_test['id']

In [13]:
params_2nd = {'bagging_fraction': 0.8999882607358867,
 'bagging_freq': int(95.0),
 'feature_fraction': 0.2570109385381975,
 'learning_rate': 0.13601832720254403,
 'max_depth': int(26.0),
 'num_leaves': int(175.0),
 'test_size': 0.08363501292068126}

lgb_m_2nd = LightGBMWrapper(**params_2nd)
lgb_m_2nd.fit(utils.filtrar_features(df_train_s, features + ['stack01', 'stack02']), df_train['precio'].values)

LightGBMWrapper(bagging_fraction=0.8999882607358867, bagging_freq=95,
                boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                feature_fraction=0.2570109385381975, importance_type='split',
                learning_rate=0.13601832720254403, max_depth=26,
                min_child_samples=20, min_child_weight=0.001,
                min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=175,
                objective=None, random_state=None, reg_alpha=0.0,
                reg_lambda=0.0, silent=True, subsample=1.0,
                subsample_for_bin=200000, subsample_freq=0,
                test_size=0.08363501292068126)

In [ ]:
df_test_s['target'] = lgb_m_2nd.predict(utils.filtrar_features(df_test_s, features + ['stack01', 'stack02']))
df_test_s[['id', 'target']].to_csv('respuesta26.csv', index = False)

## Prediccion solo con features de stacking

In [18]:
params_2nd = {'bagging_fraction': 0.8924398062087346,
 'bagging_freq': int(36.0),
 'feature_fraction': 0.16167385124183287,
 'learning_rate': 0.054693418899570134,
 'max_depth': int(4.0),
 'num_leaves': int(93.0)}
keras_mae_train = utils.MAE(y_train, lgb_m_2nd.predict(stack.transform(x_train)))
keras_mae_test = utils.MAE(y_test, lgb_m_2nd.predict(stack.transform(x_test)))
print(f"MAE Stacking (train): {keras_mae_train:.5f}")
print(f"MAE Stacking (test): {keras_mae_test:.5f}")
lgb_m_2nd = LightGBMWrapper(**params_2nd)
lgb_m_2nd.fit(stack.transform(utils.filtrar_features(df_train_f.drop('precio', axis=1), features)), df_train_f['precio'].values)

Transforming...

estimator  0: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done


KeyboardInterrupt: 

In [ ]:
keras_mae_train = utils.MAE(y_train, lgb_m_2nd.predict(stack.transform(x_train)))
keras_mae_test = utils.MAE(y_test, lgb_m_2nd.predict(stack.transform(x_test)))
print(f"MAE Stacking (train): {keras_mae_train:.5f}")
print(f"MAE Stacking (test): {keras_mae_test:.5f}")

In [ ]:
s_test_f = stack.transform(utils.filtrar_features(df_test_f, features))
y_pred_test_f = lgb_m_2nd.predict(s_test_f)
df_test_f['target'] = y_pred_test_f
df_test_f[['id', 'target']].to_csv('respuesta21.csv', index = False)

In [ ]:
features = ['stack01', 'stack02', 'stack03']

def eval_lightgbm(args):
    num_leaves, learning_rate, feature_fraction, bagging_fraction, bagging_freq, max_depth = args

    lgb_train = lgb.Dataset(s_train, y_train)
#     lgb_eval = lgb.Dataset(s_test, y_test, reference=lgb_train)
    
    num_leaves = int(num_leaves)
    bagging_freq = int(bagging_freq)
    max_depth = int(max_depth)
    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'mae'}, # Si se deja vacio se toma el ideal para llegar al 'objective'
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': bagging_freq,
        'max_depth': max_depth,
        'verbose': -1,
    }

    gbm = lgb.train(params,
                    lgb_train,
#                     valid_sets=lgb_eval,
                    num_boost_round=250,
#                     early_stopping_rounds=15,
                    verbose_eval=-1)
    
    y_pred_test = gbm.predict(s_test, num_iteration=gbm.best_iteration)
    return utils.MAE(y_test, y_pred_test)

space = [hp.quniform('num_leaves', 30, 130, 1), hp.uniform('learning_rate', 0.05, 0.9),
        hp.uniform('feature_fraction', 0.10, 0.90), hp.uniform('bagging_fraction', 0.10, 0.90),
        hp.quniform('bagging_freq', 1, 130, 1), hp.quniform('max_depth', 1, 20, 1)]

hps = fmin(eval_lightgbm, space=space, algo=tpe.suggest, max_evals=100, verbose=1)

display(hps)

In [ ]:

# keras_mae_train = utils.MAE(y_test, lgb_m.predict(x_test_s))
# print(f"MAE Keras (train): {keras_mae_train:.5f}")